<a href="https://colab.research.google.com/github/PyotrAndreev/Math/blob/main/power_iteration_and_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Compute The Dominant Eigenvector & Eigenvalue

In [2]:
### Use Only For Compute The Dominant Eigenvector & Eigenvalue ###
import numpy as np

def get_dominant_eigenvalue_and_eigenvector(data, num_steps):
    """
    data: np.ndarray – symmetric diagonalizable real-valued matrix
    num_steps: int – number of power method steps
    
    Returns:
    eigenvalue: float – dominant eigenvalue estimation after `num_steps` steps
    eigenvector: np.ndarray – corresponding eigenvector estimation
    """
    ### YOUR CODE HERE
    A = data  # matrix A
    b0 = np.ones(data.shape[0])
    b0 = b0 / np.linalg.norm(b0)  # L2

    b = b0
    for _ in range(num_steps):
      Ab = A @ b
      b = Ab / np.linalg.norm(Ab)  # L2
    λ = b.T @ Ab

    return float(λ), b

## Visualise The Convergence Of Dominant Eigenvector & Eigenvalue

In [3]:
def get_dominant_eigens(matrix: np.ndarray, start_vector: np.ndarray, num_steps: int) -> dict[int, dict[str, np.ndarray or float]]:
    A = matrix  # matrix A
    b = start_vector.astype(float)
    assert A.shape[0] == b.shape[0], "the matrix.shape[0] & start_vector.shape[0] must be equal"

    b_normed = b / np.linalg.norm(b)  # L2

    eigens_by_steps: dict = {}
    eigens_noemed_by_steps: dict = {}

    for i in range(num_steps):
      Ab = A @ b
      λ = (b.T @ Ab) / (b.T @ b)
      eigens_by_steps[i] = {'vector': b, 'value': λ}
      b = Ab

      Ab_normed = A @ b_normed
      λ_normed = (b_normed.T @ Ab_normed) / (b_normed.T @ b_normed)
      eigens_noemed_by_steps[i] = {'vector': b_normed, 'value': λ_normed}
      b_normed = Ab_normed / np.linalg.norm(Ab_normed)  # L2

    return eigens_by_steps, eigens_noemed_by_steps


# Example usage
A = np.array([[4, -2],
              [1,  1]])

eigens_by_steps, eigens_noemed_by_steps = get_dominant_eigens(A, start_vector=np.array([1, 1]), num_steps=20)
print("Eigens:", eigens_by_steps, eigens_noemed_by_steps)

Eigens: {0: {'vector': array([1., 1.]), 'value': 2.0}, 1: {'vector': array([2., 2.]), 'value': 2.0}, 2: {'vector': array([4., 4.]), 'value': 2.0}, 3: {'vector': array([8., 8.]), 'value': 2.0}, 4: {'vector': array([16., 16.]), 'value': 2.0}, 5: {'vector': array([32., 32.]), 'value': 2.0}, 6: {'vector': array([64., 64.]), 'value': 2.0}, 7: {'vector': array([128., 128.]), 'value': 2.0}, 8: {'vector': array([256., 256.]), 'value': 2.0}, 9: {'vector': array([512., 512.]), 'value': 2.0}, 10: {'vector': array([1024., 1024.]), 'value': 2.0}, 11: {'vector': array([2048., 2048.]), 'value': 2.0}, 12: {'vector': array([4096., 4096.]), 'value': 2.0}, 13: {'vector': array([8192., 8192.]), 'value': 2.0}, 14: {'vector': array([16384., 16384.]), 'value': 2.0}, 15: {'vector': array([32768., 32768.]), 'value': 2.0}, 16: {'vector': array([65536., 65536.]), 'value': 2.0}, 17: {'vector': array([131072., 131072.]), 'value': 2.0}, 18: {'vector': array([262144., 262144.]), 'value': 2.0}, 19: {'vector': array([

## Compute Eigenvectors & Eigenvalues By Analytic Method (?)

In [4]:
def get_eigens_characteristic_polynomial_method(matrix: np.ndarray) -> dict[int, dict[str, np.ndarray or float]]:
    """
    Compute the eigenvalues and eigenvectors of a square matrix using the characteristic polynomial method.

    Parameters:
    -----------
    matrix : np.ndarray
        A square matrix for which the eigenvalues and eigenvectors are to be computed.

    Returns:
    --------
    dict[int, dict[str, np.ndarray or float]]
        A dictionary where the keys are integer indices and the values are dictionaries.
        Each inner dictionary has two key-value pairs:
        - 'value': eigenvalue (float)
        - 'vector': corresponding eigenvector (np.ndarray)

    Raises:
    -------
    AssertionError
        If the input matrix is not square.

    Example:
    --------
    >>> A = np.array([[4, -2], [1, 1]])
    >>> eigens = get_eigens_characteristic_polynomial_method(A)
    >>> print(eigens[0]['value'])
    3.0
    >>> print(eigens[0]['vector'])
    [0.89442719 0.4472136 ]

    Note:
    -----
    The function uses the Singular Value Decomposition (SVD) to find the null space of the matrix (A - λI),
    which gives the eigenvector corresponding to the eigenvalue λ.
    """

    A = matrix  # matrix A
    # Ensure A is a square matrix
    assert A.shape[0] == A.shape[1], "Matrix should be square"

    # Compute eigenvalues using the characteristic polynomial
    eigenvalues = np.linalg.eigvals(A)

    eigens_analytical: dict = {}

    for i, λ in enumerate(eigenvalues):
        # Solve (A - λ*I) x = 0 for each eigenvalue
        temp_matrix = A - λ * np.identity(A.shape[0])

        # Find null space of the temp_matrix to get the eigenvector
        eigenvector = np.linalg.svd(temp_matrix)[-1][-1]

        eigens_analytical[i] = {'value': λ, 'vector': eigenvector}

    return eigens_analytical


# Example usage
A = np.array([[4, -2],
              [1,  1]])

eigens_analytical = get_eigens_characteristic_polynomial_method(A)
print("Eigens:", eigens_analytical)


Eigens: {0: {'value': 3.0, 'vector': array([0.89442719, 0.4472136 ])}, 1: {'value': 2.0, 'vector': array([0.70710678, 0.70710678])}}


## Visualize The Convergens

In [8]:
# !pip install bokeh

In [9]:
from bokeh.layouts import column, row
from bokeh.models import Slider, ColumnDataSource, CustomJS, HoverTool, Div
from bokeh.plotting import figure, show

from bokeh.io import curdoc  # theme color
from bokeh.io import output_notebook # activate Bokeh output in Jupyter notebook

output_notebook()

Loading BokehJS ...

In [10]:
curdoc().theme = "dark_minimal"
# curdoc().theme = None


def visualize_power_method(matrix: np.array, start_vector: np.array, num_steps: int) -> None:
  header_div = Div(
      text="""
  <h1>Power Iterati/Method Visualization</h1>
  <p>More Theory: <a href="https://t.me/+g6swicRrEVEyNTAy" target="_blank"> Petr | ML notes </a></p> target="_blank">
  """,
      sizing_mode="stretch_width",
  )
  show(header_div)

  A = matrix
  eigens_by_steps, eigens_normed_by_steps = get_dominant_eigens(matrix, start_vector, num_steps)
  eigens_analytical = get_eigens_characteristic_polynomial_method(matrix)

  v1 = [list(iter['vector']) for iter in eigens_by_steps.values()]  # dominant_eigenvector_approach
  v2 = [list(iter['vector']) for iter in eigens_normed_by_steps.values()]  # normed_dominant_eigenvector_approach

  v1_true = eigens_analytical[0]['vector']
  v2_true = eigens_analytical[1]['vector']

  # Compute λ residual
  λ_max = max(iter['value'] for iter in eigens_analytical.values())
  error_1 = abs(np.array([iter['value'] for iter in eigens_by_steps.values()]) - λ_max)
  error_2 = abs(np.array([iter['value'] for iter in eigens_normed_by_steps.values()]) - λ_max)

  # Create data sources for the plots
  source1 = ColumnDataSource(data=dict(x=[0, v1[0][0]], y=[0, v1[0][1]]))
  source2 = ColumnDataSource(data=dict(x=[0, v2[0][0]], y=[0, v2[0][1]]))
  source_diff1 = ColumnDataSource(data=dict(x=list(range(1)), y=error_1[:1]))
  source_diff2 = ColumnDataSource(data=dict(x=list(range(1)), y=error_2[:1]))

  # Create the vector visualization plot
  p1 = figure(width=700, height=300, title='Vector Comparison By Iteration',
              x_axis_label='x', y_axis_label='y',
              x_range=(-2, 2), y_range=(-2, 2))

  p1.line([0, v1_true[0]], [0, v1_true[1]], line_width=2, line_color="green",
          legend_label="eigenvector v1", line_dash="dotted")

  p1.line([0, v2_true[0]], [0, v2_true[1]], line_width=2, line_color="green",
          legend_label="eigenvector v2", line_dash="dotted")

  p1.line('x', 'y', source=source1, line_width=2, line_color="blue", alpha=0.5,
          legend_label="aproach v1")

  p1.line('x', 'y', source=source2, line_width=2, line_color="red", alpha=0.5,
          legend_label="normed aproach v2")

  # Create a plot for λ residual
  p2 = figure(width=700, height=300, title='Eigenvalue Error',
              x_axis_label='iteration', y_axis_label='∆λ',
              x_range=(0, len(v1)), y_range=(min(error_2)-1, max(error_2)+1))

  p2.line('x', 'y', source=source_diff1, line_width=3, line_color="blue",
          alpha=0.8, legend_label="∆λ")

  p2.line('x', 'y', source=source_diff2, line_width=2, line_color="red",
          alpha=1, legend_label="∆λ normed")

  # JavaScript callback
  callback = CustomJS(args=dict(source1=source1, source2=source2, source_diff1=source_diff1, source_diff2=source_diff2, vectors1=v1, vectors2=v2, differences1=error_1, differences2=error_2), code="""
      const data1 = source1.data;
      const data2 = source2.data;
      const data_diff1 = source_diff1.data;
      const data_diff2 = source_diff2.data;
      const selected_vector1 = vectors1[cb_obj.value];
      const selected_vector2 = vectors2[cb_obj.value];
      data1['x'][1] = selected_vector1[0];
      data1['y'][1] = selected_vector1[1];
      data2['x'][1] = selected_vector2[0];
      data2['y'][1] = selected_vector2[1];
      data_diff1['x'] = Array.from({length: cb_obj.value+1}, (_, i) => i);
      data_diff1['y'] = differences1.slice(0, cb_obj.value+1);
      data_diff2['x'] = Array.from({length: cb_obj.value+1}, (_, i) => i);
      data_diff2['y'] = differences2.slice(0, cb_obj.value+1);
      source1.change.emit();
      source2.change.emit();
      source_diff1.change.emit();
      source_diff2.change.emit();
  """)

  # Add a slider
  slider = Slider(start=0, end=len(v1)-1, value=0, step=1, title="Iteration")
  slider.js_on_change('value', callback)

  layout = column(slider, column(p1, p2))
  show(layout)

1. The results it is not right in every cases. I still have not found the mistakes. I let it for the future.
2. It may be better if the program is run in Colab.

In [12]:
matrix = np.array([[0, 0], [0, 0]])
start_vector = np.array([-10, -100])
num_steps = 15

visualize_power_method(matrix, start_vector, num_steps)

/tmp/ipykernel_129028/3835903271.py:20: RuntimeWarning: invalid value encountered in divide
  b_normed = Ab_normed / np.linalg.norm(Ab_normed)  # L2
/tmp/ipykernel_129028/3835903271.py:13: RuntimeWarning: invalid value encountered in scalar divide
  λ = (b.T @ Ab) / (b.T @ b)
